# Crypto Portfolio Optimization

Goal: Create a machine learning-driven portfolio optimization system for cryptocurrencies, dynamically adjusting allocations to maximize returns and minimize risks based on predictive analytics.

### Top crypto cryptocurrencies 

In [40]:
import requests
import pandas as pd

In [71]:
def fetch_top_coins_tickers(vs_currency='usd', limit=30):
    url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    params = {
        'vs_currency': vs_currency,
        'order': 'market_cap_desc',
        'per_page': limit,
        'page': 1,
        'sparkline': False,
        'price_change_percentage': '24h'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data: Status code {response.status_code}")
        return []

    data = response.json()

##Extract tickers
    tickers = [coin['symbol'].upper() for coin in data]
    return tickers

In [72]:
top_coins = fetch_top_coins_tickers(vs_currency='usd', limit=30)
print(top_coins)

['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'STETH', 'USDC', 'ADA', 'AVAX', 'TRX', 'DOGE', 'LINK', 'DOT', 'MATIC', 'WBTC', 'TON', 'ICP', 'SHIB', 'UNI', 'LTC', 'BCH', 'DAI', 'IMX', 'FIL', 'HBAR', 'KAS', 'LEO', 'TAO', 'ATOM']


In [49]:
#&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc

In [78]:
import requests
import pandas as pd

def fetch_historical_data(coin_id='bitcoin', vs_currency='usd', days='max', interval='daily'):
    url = f'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': interval
    }
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"Error fetching data for {coin_id}: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

    data = response.json()

    # Check the type of data and print it out to understand the structure
    print(type(data))
    print(data)

    # Assuming the correct structure is returned, proceed to parse the data
    prices = data['prices'] if 'prices' in data else []
    volumes = data['total_volumes'] if 'total_volumes' in data else []
    market_caps = data['market_caps'] if 'market_caps' in data else []

    if not prices or not volumes or not market_caps:
        print(f"No data found for {coin_id}")
        return pd.DataFrame()  # Return an empty DataFrame if there's no data

    # Convert lists to DataFrame
    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Volume'] = pd.Series([volume[1] for volume in volumes])
    df['Market Cap'] = pd.Series([market_cap[1] for market_cap in market_caps])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    
    return df

# Rest of your code to iterate over coins and fetch data...


# Assuming 'top_coins' is a list of coin IDs
top_coins = ['eth', 'xrp']  # Example list, replace with your actual list

# Fetch historical data for each coin
historical_data = {}
for coin_id in top_coins:
    print(f"Fetching data for {coin_id}...")
    df = fetch_historical_data(coin_id)
    historical_data[coin_id] = df


Fetching data for eth...
<class 'list'>
[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 51637, 'market_cap': 1013232543847, 'market_cap_rank': 1, 'fully_diluted_valuation': 1083684305457, 'total_volume': 27852463030, 'high_24h': 51992, 'low_24h': 50706, 'price_change_24h': 585.17, 'price_change_percentage_24h': 1.14624, 'market_cap_change_24h': 7507885097, 'market_cap_change_percentage_24h': 0.74651, 'circulating_supply': 19634762.0, 'total_supply': 21000000.0, 'max_supply': 21000000.0, 'ath': 69045, 'ath_change_percentage': -25.40609, 'ath_date': '2021-11-10T14:24:11.849Z', 'atl': 67.81, 'atl_change_percentage': 75853.32484, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2024-02-22T17:38:06.127Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://assets.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price'

In [77]:
historical_data

{'ethereum': Empty DataFrame
 Columns: []
 Index: [],
 'litecoin': Empty DataFrame
 Columns: []
 Index: []}

In [44]:
import requests
import pandas as pd

def fetch_historical_data(coin_id='bitcoin', vs_currency='usd', days='max', interval='daily'):
    url = f'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': interval
    }
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"Error fetching data for {coin_id}: Status code {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame on error
    
    data = response.json()
    
    # Print raw data for inspection
    print(f"Raw data for {coin_id}: {data}")
    
    if 'prices' not in data or 'total_volumes' not in data or 'market_caps' not in data:
        print(f"Missing data for {coin_id}.")
        return pd.DataFrame()  # Return an empty DataFrame if keys are missing
    
    prices = data['prices']
    volumes = data['total_volumes']
    market_caps = data['market_caps']
    
    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Volume'] = pd.DataFrame(volumes, columns=['Timestamp', 'Volume'])['Volume']
    df['Market Cap'] = pd.DataFrame(market_caps, columns=['Timestamp', 'Market Cap'])['Market Cap']
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    
    return df


In [45]:

# Fetch historical data for each coin
historical_data = {}
for coin_id in top_coins:
    print(f"Fetching data for {coin_id}...")
    df = fetch_historical_data(coin_id)
    historical_data[coin_id] = df


Fetching data for bitcoin...
Raw data for bitcoin: [{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 51669, 'market_cap': 1012692727812, 'market_cap_rank': 1, 'fully_diluted_valuation': 1083110761347, 'total_volume': 28540761405, 'high_24h': 51992, 'low_24h': 50706, 'price_change_24h': 565.75, 'price_change_percentage_24h': 1.10708, 'market_cap_change_24h': 9307687716, 'market_cap_change_percentage_24h': 0.92763, 'circulating_supply': 19634693.0, 'total_supply': 21000000.0, 'max_supply': 21000000.0, 'ath': 69045, 'ath_change_percentage': -25.29962, 'ath_date': '2021-11-10T14:24:11.849Z', 'atl': 67.81, 'atl_change_percentage': 75961.73653, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2024-02-22T16:08:17.427Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://assets.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'cur

In [51]:
historical_data

{'bitcoin': Empty DataFrame
 Columns: []
 Index: [],
 'ethereum': Empty DataFrame
 Columns: []
 Index: [],
 'tether': Empty DataFrame
 Columns: []
 Index: [],
 'binancecoin': Empty DataFrame
 Columns: []
 Index: [],
 'solana': Empty DataFrame
 Columns: []
 Index: [],
 'ripple': Empty DataFrame
 Columns: []
 Index: [],
 'staked-ether': Empty DataFrame
 Columns: []
 Index: [],
 'usd-coin': Empty DataFrame
 Columns: []
 Index: [],
 'cardano': Empty DataFrame
 Columns: []
 Index: [],
 'avalanche-2': Empty DataFrame
 Columns: []
 Index: [],
 'tron': Empty DataFrame
 Columns: []
 Index: [],
 'dogecoin': Empty DataFrame
 Columns: []
 Index: [],
 'chainlink': Empty DataFrame
 Columns: []
 Index: [],
 'polkadot': Empty DataFrame
 Columns: []
 Index: [],
 'matic-network': Empty DataFrame
 Columns: []
 Index: [],
 'wrapped-bitcoin': Empty DataFrame
 Columns: []
 Index: [],
 'the-open-network': Empty DataFrame
 Columns: []
 Index: [],
 'internet-computer': Empty DataFrame
 Columns: []
 Index: [],


In [32]:
import requests
import pandas as pd

def fetch_historical_data(coin_id='bitcoin', vs_currency='usd', days='max', interval='daily'):
    # Corrected URL
    url = f'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    
    
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': interval
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    prices = data['prices']  # List of [timestamp, price]
    volumes = data['total_volumes']  # List of [timestamp, volume]
    market_caps = data['market_caps']  # List of [timestamp, market_cap]
    
    # Convert to DataFrame for easier handling
    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Volume'] = pd.DataFrame(volumes, columns=['Timestamp', 'Volume'])['Volume']
    df['Market Cap'] = pd.DataFrame(market_caps, columns=['Timestamp', 'Market Cap'])['Market Cap']
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    
    return df

# Assuming 'top_coins' is a list of coin IDs
top_coins = ['bitcoin', 'ethereum', 'litecoin']  # Example list, replace with your actual list

# Fetch historical data for each coin
historical_data = {}
for coin_id in top_coins:
    print(f"Fetching data for {coin_id}...")
    df = fetch_historical_data(coin_id)
    historical_data[coin_id] = df


Fetching data for bitcoin...


TypeError: list indices must be integers or slices, not str

In [1]:
import requests



# List of cryptocurrencies to search for
cryptos = [
    'bitcoin', 'ethereum', 'tether', 'binancecoin', 'solana', 'ripple',
    'staked-ether', 'usd-coin', 'cardano', 'avalanche-2', 'tron', 'dogecoin',
    'chainlink', 'polkadot', 'matic-network', 'wrapped-bitcoin', 'the-open-network',
    'internet-computer', 'shiba-inu', 'uniswap', 'bitcoin-cash', 'litecoin',
    'dai', 'immutable-x', 'kaspa', 'cosmos', 'leo-token', 'bittensor',
    'blockstack', 'ethereum-classic'
]

def get_crypto_ticker(crypto_id):
    """Fetches the ticker symbol for a given cryptocurrency using the CoinGecko API."""
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an error for bad responses
        data = response.json()
        return data.get('symbol', 'N/A').upper()
    except requests.RequestException as e:
        print(f"Error fetching data for {crypto_id}: {e}")
        return None

tickers = {crypto: get_crypto_ticker(crypto) for crypto in cryptos}

for crypto, ticker in tickers.items():
    print(f"{crypto}: {ticker}")


Error fetching data for ripple: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/ripple
Error fetching data for staked-ether: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/staked-ether
Error fetching data for usd-coin: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/usd-coin
Error fetching data for cardano: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/cardano
Error fetching data for avalanche-2: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/avalanche-2
Error fetching data for tron: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/tron
Error fetching data for dogecoin: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/dogecoin
Error fetching data for chainlink: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v

In [48]:
import yfinance as yf
import pandas as pd

symbols = [f"{top_coins}-USD" for top_coins in top_coins]

# Initialize an empty DataFrame for concatenating all cryptocurrency data
all_data = pd.DataFrame()

# Specify the date range
start_date = '2021-01-01'
end_date = '2024-02-21'

for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date)
    if not data.empty:
        data['Symbol'] = symbol.replace('-USD', '')  # Add a 'Symbol' column to identify the cryptocurrency
        all_data = pd.concat([all_data, data], axis=0)

# Reset the index of the concatenated DataFrame
all_data.reset_index(inplace=True)



Fetching data for bitcoin-USD...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for ethereum-USD...
Fetching data for tether-USD...


[*********************100%%**********************]  1 of 1 completed


Fetching data for binancecoin-USD...


...

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BITTENSOR-USD']: Exception('%ticker%: No timezone found, symbol may be delisted')


Fetching data for blockstack-USD...


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BLOCKSTACK-USD']: Exception('%ticker%: No timezone found, symbol may be delisted')


Fetching data for ethereum-classic-USD...


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ETHEREUM-CLASSIC-USD']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [50]:
all_data

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2021-12-10,0.000000,0.000000,0.000000,0.000000,0.000000,3326077,bitcoin
1,2021-12-11,0.000000,0.000000,0.000000,0.000000,0.000000,3445750,bitcoin
2,2021-12-12,0.000000,0.000000,0.000000,0.000000,0.000000,1941490,bitcoin
3,2021-12-13,0.000000,0.000000,0.000000,0.000000,0.000000,1014266,bitcoin
4,2021-12-14,0.000000,0.000000,0.000000,0.000000,0.000000,461149,bitcoin
...,...,...,...,...,...,...,...,...
3414,2023-12-27,1.000424,1.001485,0.998941,1.000313,1.000313,264968316,dai
3415,2023-12-28,1.000070,1.001127,0.997704,0.999737,0.999737,293430251,dai
3416,2023-12-29,0.999811,1.000834,0.997398,1.000419,1.000419,244283931,dai
3417,2023-12-30,1.000409,1.000799,0.998255,0.999455,0.999455,214916196,dai


In [27]:
import requests
import pandas as pd

def fetch_historical_data(coin_id='bitcoin', vs_currency='usd', days='max', interval='daily'):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': interval
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    prices = data['prices']  # List of [timestamp, price]
    volumes = data['total_volumes']  # List of [timestamp, volume]
    market_caps = data['market_caps']  # List of [timestamp, market_cap]
    
    # Convert to DataFrame for easier handling
    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Volume'] = pd.DataFrame(volumes)[1]
    df['Market Cap'] = pd.DataFrame(market_caps)[1]
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    
    return df

# Example usage
df = fetch_historical_data()


KeyError: 'prices'

In [6]:
df

,Timestamp,Price,Volume,Market Cap
0,2013-04-28 00:00:00,135.300000,0.000000e+00,1.500518e+09
1,2013-04-29 00:00:00,141.960000,0.000000e+00,1.575032e+09
2,2013-04-30 00:00:00,135.300000,0.000000e+00,1.501657e+09
3,2013-05-01 00:00:00,117.000000,0.000000e+00,1.298952e+09
4,2013-05-02 00:00:00,103.430000,0.000000e+00,1.148668e+09
...,...,...,...,...
3945,2024-02-17 00:00:00,52166.434310,2.485043e+10,1.024019e+12
3946,2024-02-18 00:00:00,51684.517493,1.985188e+10,1.015778e+12
3947,2024-02-19 00:00:00,52138.485789,1.702588e+10,1.023586e+12
3948,2024-02-20 00:00:00,51764.313185,2.255512e+10,1.016146e+12
